<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex19-Fundamental_Law.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investments: Theory, Fundamental Analysis, and Data Driven Analytics**, Bates, Boyer, and Fletcher

# Example Chapter 19: The Fundamental Law of Asset Management
In this example we create a simulation to show that the Fundamental Law of Active Management works.

### Imports and Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Define Simulation Parameters
We first define `b`, which represents the sensitivity of each asset to its signal. We then define `sigma_z` rhich is the volatility of the residuals in the abnormal return regression. We then define a lisst of numbers which represent various values for breadth.  The parameter `n_years` is the number of years over which we run the simulation. We then create a random number generator object to gnerate random samples of numbers.

In [ ]:
b = 0.10
sigma_z= 0.30
breadth_values = [1, 5, 25, 100, 400, 1600]
n_years = 10000
# Create a random number generator object
rng = np.random.default_rng()


### Simulate Signals and Abnormal Returns
Here we create a function `simulate_ir()`. We first simulate three "blocks" of numbers.  The number of rows in each block is `n_years`. Each row represents a different year. The number of columns of each block is a chose value for **breadth**, which represents both the number of signals we have, and the number of assets. We first simulate a blocks of signals. We then simulate a block of residuals with mean zero and volatility given by `sigma_z`.  Then using these signals, we create a block of abnormal returns, $a_{i,y}$ where the abnormla return for asset $i$ in year $y$ is   
\begin{equation}
a_{iy}=bf_{i,y}+z_{i,y}.  
\end{equation}
Next, we create a block of active weights by scaling the signals by a scalar, $c$. Because the signals are mean zero, the active weights sum to approximately zero. Any small imbalance is absorbed by a position in the risk-free asset, which contributes neither abnormal return nor volatility. We then multiply each element in the block of abnormal returns by each weight element, and add these up across columns using `axis=1`. This gives us a column of abnormal returns for the fund portfolio in each year. Finally, we compute the information ratio. We estimate the expected abnormal return, or alpha, as the average portfolio abnormal return over all years. We estimate active risk as the standard deviation of the abnormal return.

####Function: `simulate_ir()`  
**Inputs**
* `b`: the sensitivity of each asset to its signal.  
* `BR`: breadth
* `n_years`: the number of years to create abnormal returns
* `sigma_e`: the volatility of the residual in the abnormal return regression

**Output**
*  `IR`: the simulated information ratio.  


In [ ]:
def simulate_ir(b, BR, n_years, sigma_e):
    # simulate signals and realized outcomes for abnormal returns

    # signals are mean zero, with volatility=1.0
    f = rng.normal(loc=0, scale=1, size=(n_years, BR))

    # residulas are mean zero, with volatility=sigma_e
    z = rng.normal(loc=0, scale=sigma_e, size=(n_years, BR))

    # create abnormal returns
    a = b * f +  z

    # choose scalaing constant for active weights
    c = .5
    h = c * f

    # portfolio abnormal return per year as a weighted sum
    port_abnorm_ret = (h * a).sum(axis=1)

    # information ratio
    IR = port_abnorm_ret.mean() / port_abnorm_ret.std()

    return IR

### Run Simulation Experiment
For each value of breadth in `breadth_values` we simulate abnormal returns and use these to estimate the information ratio using the function `simulate_ir()` above.  We then compute the $R^2$ in the regression of abnormal returns on the signal.  The square-root of the R-squared is the correlation between each abnormal return and the signal, `IC`. We then compute the information ratio as stated by the **Fundamental Law of Active Management**.  Finally, we print out the value of breadth, the simulated IR, and the IR as given by the **Fundamental Law**.   

In [ ]:
# run experiments
for BR in breadth_values:
    ir = simulate_ir(b, BR, n_years, sigma_z)
    Rsquared=(b ** 2)/(b ** 2 + sigma_z ** 2)
    IC = np.sqrt(Rsquared)
    theory = IC * np.sqrt(BR)
    print(f"BR={BR:4d}  Simulated IR={ir:5.3f}  Theory={theory:5.3f}")
